In [3]:
# load gt
import json

path = '../../dataset/qrecc-test.json'

with open(path) as f: 
    data = json.load(f)

In [19]:
# load all samples with model labels
import os

path = '../../runs/'

samples = {}
for filename in os.listdir(path):
    if filename.endswith('.json'):
        # load model run
        with open(path+filename, encoding="utf-8") as f:
            print(filename)
            model_label = filename.split('.')[0]
            # load samples for this run
            run = json.load(f)
            for sample in run:
                sample_id = "%d_%d" % (sample['Conversation_no'], sample['Turn_no']) 
                if sample_id not in samples:
                    samples[sample_id] = {}
                samples[sample_id][model_label] = sample['Model_answer']
                
print('\n %d samples loaded' % len(samples))

simple_baseline_rewritten.json
gpt3_run.json
scai-qrecc21-naacl-baseline.json
basic-baseline-run.json

 16736 samples loaded


In [20]:
# load answer encoder
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-MiniLM-L12-v2')

In [21]:
# create a single ranked list
ranked_list = []
for i, sample in enumerate(data[:500]):  #[:limit]
    # group all answers
    gt_answers = [sample['Answer']]
    sample_id = "%d_%d" % (sample['Conversation_no'], sample['Turn_no']) 
    
    if sample_id in samples:
        model_answers = list(samples[sample_id].values())
        model_labels = list(samples[sample_id].keys())
    
        #Compute embedding for both lists
        gt_embeddings = model.encode(gt_answers, convert_to_tensor=True)
        model_embeddings = model.encode(model_answers, convert_to_tensor=True)

        #Compute cosine-similarits
        cosine_scores = util.pytorch_cos_sim(gt_embeddings, model_embeddings)
        for i in range(len(cosine_scores)):
            for j in range(len(cosine_scores[i])):
                ranked_list.append({'question': sample['Rewrite'],
                                    'gt': sample['Answer'],
                                    'model': model_labels[j],
                                    'predicted': model_answers[j],
                                    'score': cosine_scores[i][j]})

In [22]:
#Sort scores in decreasing order
ranked_list = sorted(ranked_list, key=lambda x: x['score'], reverse=True)

limit = 20
for sample in ranked_list[:limit]:
    print("{} \t {} \t {} \t {} \t {:.4f}".format(sample['question'], sample['gt'], sample['predicted'], sample['model'], sample['score']))

What are scalable data center network architectures? 	 DCNs need to be scalable and efficient to connect tens or even hundreds of thousands of servers to handle the growing demands of Cloud computing. 	 DCNs need to be scalable and efficient to connect tens or even hundreds of thousands of servers to handle the growing demands of Cloud computing 	 scai-qrecc21-naacl-baseline 	 0.9917
Can you milk Angora goats? 	 Angora mothers have been known to not produce enough milk if they give birth to more than 1. 	 Angora mothers have been known to not produce enough milk if they give birth to more than 	 scai-qrecc21-naacl-baseline 	 0.9791
What is weathering? 	 Weathering is the breaking down of rocks, soil, and minerals as well as wood and artificial materials through contact with the Earth's atmosphere, water, and biological organisms. 	 Weathering is breaking down rocks, soil, and minerals as well as wood and artificial materials by contacting the atmosphere, water, and biological organisms